In [ ]:
%load_ext autoreload
%autoreload 2


In [ ]:
import csv
import matplotlib.pyplot as plt
import multiprocessing
import numpy as np
import seaborn as sns

from lieroy import se3
from recova.clustering import CenteredClusteringAlgorithm, RegistrationPairClusteringAdapter
from recova.covariance import SamplingCovarianceComputationAlgorithm, SamplingDistributionComputationAlgorithm
from recova.registration_result_database import RegistrationPairDatabase

In [ ]:
db = RegistrationPairDatabase('/home/dlandry/dataset/db_eth_05/')
pairs = db.registration_pairs()

In [ ]:
clustering_algo = CenteredClusteringAlgorithm(radius=0.005, k=20, n_seed_init=20)
clustering_algo.seed_selector = 'localized'
clustering = RegistrationPairClusteringAdapter(clustering_algo)

distribution_algo = SamplingDistributionComputationAlgorithm(clustering)
covariance_algo = SamplingCovarianceComputationAlgorithm(clustering_algorithm=clustering)

In [ ]:
def distance_of_one_pair(pair):
    print(pair)
    distribution = distribution_algo.compute(pair)
    
    mean, covariance = np.array(distribution['mean']), np.array(distribution['covariance'])
    
    ground_truth = pair.ground_truth()
    delta = se3.log(np.linalg.inv(ground_truth) @ mean)
    distance = np.linalg.norm(delta)

    return distance, 0.0

In [ ]:
distance_of_one_pair(pairs[5])

In [ ]:
#results = [distance_of_one_pair(x) for x in pairs]

with multiprocessing.Pool(processes=6) as pool:
    results = pool.map(distance_of_one_pair, pairs)
    

In [ ]:
covariances = [covariance_algo.compute(x) for x in pairs]

In [ ]:
cov_norms = [np.linalg.norm(x) for x in covariances]

In [ ]:
distances, _ = zip(*results)

In [ ]:
with open('/home/dlandry/distances_of_mean.csv', 'w') as f:
    writer = csv.DictWriter(f, ['location', 'reading', 'reference', 'distance', 'n_samples'])
    writer.writeheader()
    for i, pair in enumerate(pairs):
        writer.writerow({
            'location': pair.dataset,
            'reading': pair.reading,
            'reference': pair.reference,
            'distance': distances[i],
            'n_samples': 
        })

In [ ]:
sns.distplot(distances, rug=True)
plt.xlim([0.0, 0.2])
plt.show()

In [ ]:
plt.scatter(distances, cov_norms)
plt.ylim([0.0, 0.5])
plt.show()

In [ ]:
distances, closest = zip(*results)

In [ ]:
distance, closest = np.array(distances), np.array(closest)

In [ ]:
distance - closest

In [ ]:
np.max(distance - closest)

In [ ]:
distance - closest

In [ ]:
sns.distplot(closest, rug=True)
plt.xlim([0.0, 0.3])
plt.show()